# Camera capture

In [ ]:
import cv2
import os

#Comando para lista de dispositivos
#v4l2-ctl --list-devices
# Función para guardar una imagen en la carpeta especificada
def guardar_imagen(frame, folder_path, img_counter):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    img_name = os.path.join(folder_path, f"captura_{img_counter}.png")
    cv2.imwrite(img_name, frame)
    print(f"Imagen guardada: {img_name}")

# Inicialización de captura de video
vid = cv2.VideoCapture(0)

# Contador para el nombre de las imágenes
img_counter = 0

# Carpeta donde se guardarán las imágenes
folder_path = "perfect_choice/cuadro/"
while True: 
    # Capturar frame por frame
    ret, frame = vid.read()
  
    # Mostrar el frame resultante
    cv2.imshow('frame', frame)
      
    # Verificar las teclas presionadas
    key = cv2.waitKey(1) & 0xFF
    
    # Si se presiona la tecla 'q', salir del bucle
    if key == ord('q'):
        break
    
    # Si se presiona la barra espaciadora, guardar la imagen
    if key == ord(' '):
        guardar_imagen(frame, folder_path, img_counter)
        img_counter += 1

# Liberar el objeto de captura y cerrar todas las ventanas OpenCV
vid.release()
cv2.destroyAllWindows()

## Camera Calibration

In [4]:
import cv2
import numpy as np
import os
import glob
 
# Defining the dimensions of checkerboard
CHECKERBOARD = (6,9)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 
# Extracting path of individual image stored in a given directory
images = glob.glob('astra/cuadro/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[488.14038913   0.         319.45549715]
 [  0.         486.90461255 237.52747167]
 [  0.           0.           1.        ]]
dist : 

[[-0.10861338  0.51260148 -0.00627479 -0.00183073 -0.65896048]]
rvecs : 

(array([[-0.43059584],
       [-0.37931028],
       [-1.53375766]]), array([[-0.009669  ],
       [-0.73348296],
       [-1.73000241]]), array([[ 0.02156296],
       [-0.02551099],
       [-1.53768858]]), array([[-0.19363144],
       [ 0.08932663],
       [-1.56623356]]), array([[ 0.1856273 ],
       [-0.47805443],
       [-1.55633692]]), array([[-0.34298687],
       [-0.4090071 ],
       [-1.59125593]]), array([[ 0.22151906],
       [-0.12392557],
       [-1.41615283]]), array([[ 0.25842664],
       [-0.3312131 ],
       [-1.42878996]]), array([[-0.14546338],
       [ 0.00361596],
       [-1.51186433]]), array([[-0.36029751],
       [-0.25883089],
       [-1.4951274 ]]), array([[-0.41978312],
       [ 0.00192161],
       [-1.49769548]]), array([[-0.37449098],
 

## Circle calibration

In [1]:
import numpy as np
import cv2
import os
import glob
#import yam

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

########################################Blob Detector##############################################

# Setup SimpleBlobDetector parameters.
blobParams = cv2.SimpleBlobDetector_Params()

# Change thresholds
blobParams.minThreshold = 8
blobParams.maxThreshold = 255

# Filter by Area.
blobParams.filterByArea = True
blobParams.minArea = 64     # minArea may be adjusted to suit for your experiment
blobParams.maxArea = 2500   # maxArea may be adjusted to suit for your experiment

# Filter by Circularity
blobParams.filterByCircularity = True
blobParams.minCircularity = 0.1

# Filter by Convexity
blobParams.filterByConvexity = True
blobParams.minConvexity = 0.87

# Filter by Inertia
blobParams.filterByInertia = True
blobParams.minInertiaRatio = 0.01

# Create a detector with the parameters
blobDetector = cv2.SimpleBlobDetector_create(blobParams)

###################################################################################################

###################################################################################################

# Original blob coordinates, supposing all blobs are of z-coordinates 0
# And, the distance between every two neighbour blob circle centers is 72 centimetres
# In fact, any number can be used to replace 72.
# Namely, the real size of the circle is pointless while calculating camera calibration parameters.
objp = np.zeros((44, 3), np.float32)
objp[0]  = (0  , 0  , 0)
objp[1]  = (0  , 72 , 0)
objp[2]  = (0  , 144, 0)
objp[3]  = (0  , 216, 0)
objp[4]  = (36 , 36 , 0)
objp[5]  = (36 , 108, 0)
objp[6]  = (36 , 180, 0)
objp[7]  = (36 , 252, 0)
objp[8]  = (72 , 0  , 0)
objp[9]  = (72 , 72 , 0)
objp[10] = (72 , 144, 0)
objp[11] = (72 , 216, 0)
objp[12] = (108, 36,  0)
objp[13] = (108, 108, 0)
objp[14] = (108, 180, 0)
objp[15] = (108, 252, 0)
objp[16] = (144, 0  , 0)
objp[17] = (144, 72 , 0)
objp[18] = (144, 144, 0)
objp[19] = (144, 216, 0)
objp[20] = (180, 36 , 0)
objp[21] = (180, 108, 0)
objp[22] = (180, 180, 0)
objp[23] = (180, 252, 0)
objp[24] = (216, 0  , 0)
objp[25] = (216, 72 , 0)
objp[26] = (216, 144, 0)
objp[27] = (216, 216, 0)
objp[28] = (252, 36 , 0)
objp[29] = (252, 108, 0)
objp[30] = (252, 180, 0)
objp[31] = (252, 252, 0)
objp[32] = (288, 0  , 0)
objp[33] = (288, 72 , 0)
objp[34] = (288, 144, 0)
objp[35] = (288, 216, 0)
objp[36] = (324, 36 , 0)
objp[37] = (324, 108, 0)
objp[38] = (324, 180, 0)
objp[39] = (324, 252, 0)
objp[40] = (360, 0  , 0)
objp[41] = (360, 72 , 0)
objp[42] = (360, 144, 0)
objp[43] = (360, 216, 0)
###################################################################################################

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('astra/circle/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    keypoints = blobDetector.detect(gray) # Detect blobs.

    # Draw detected blobs as red circles. This helps cv2.findCirclesGrid() .
    im_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints_gray = cv2.cvtColor(im_with_keypoints, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findCirclesGrid(im_with_keypoints, (4,11), None, flags = cv2.CALIB_CB_ASYMMETRIC_GRID)   # Find the circle grid

    if ret == True:
        objpoints.append(objp)  # Certainly, every loop objp is the same, in 3D.

        corners2 = cv2.cornerSubPix(im_with_keypoints_gray, corners, (11,11), (-1,-1), criteria)    # Refines the corner locations.
        imgpoints.append(corners2)

        # Draw and display the corners.
        im_with_keypoints = cv2.drawChessboardCorners(img, (4,11), corners2, ret)

    cv2.imshow("img", im_with_keypoints) # display
    cv2.waitKey(0)

# When everything done, release the capture
cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[481.8392682    0.         299.85873341]
 [  0.         478.10767809 250.56980699]
 [  0.           0.           1.        ]]
dist : 

[[-0.04885269  0.16095707 -0.00069815 -0.00597664 -0.17054483]]
rvecs : 

(array([[-0.06380205],
       [-3.05531313],
       [ 0.55328964]]), array([[ 0.07790667],
       [-3.08913814],
       [ 0.66309962]]), array([[-0.01031641],
       [ 3.03059084],
       [-0.26743388]]), array([[ 0.01716334],
       [ 3.02558838],
       [-0.03447924]]), array([[-0.08928114],
       [-3.11254855],
       [-0.17510371]]), array([[-0.06239701],
       [-3.02305985],
       [-0.18003202]]), array([[-0.01736621],
       [-2.91868958],
       [-0.1118041 ]]), array([[ 1.48488109e-03],
       [-2.84896232e+00],
       [-1.10561563e-01]]), array([[ 0.05156981],
       [ 2.82962766],
       [-0.42111474]]), array([[ 0.06568944],
       [ 2.81681521],
       [-0.17264869]]), array([[ 7.72473070e-02],
       [ 2.85440288e+00],
       [-2.21173388e-04]]),

# Undistorted Image

In [6]:
import numpy as np
import cv2
folder = ('astra/cuadro/')
img = cv2.imread(folder + 'captura_10.png')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h)) # return a refined camera matrix and a Region of Interest
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imshow("Undistorted image", dst) # display
cv2.waitKey(0)